In [1]:
import pandas as pd
import numpy as np

# # Read in bank_marketing.csv as a pandas DataFrame
df = pd.read_csv('bank_marketing.csv')

In [2]:
# Split the data into three DFs
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
campaign = df[['client_id', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']]
economics = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed']]

# Rename columns
client = client.rename(columns={'client_id': 'id'})
campaign = campaign.rename(columns={
    'duration': 'contact_duration',
    'previous': 'previous_campaign_contacts',
    'y': 'campaign_outcome',
    'poutcome': 'previous_outcome',
    'campaign': 'number_contacts'
})
economics = economics.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'})

# Clean columns
client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.nan)

# Remove periods from the "job" column
client['job'] = client['job'].str.replace('.', '')

# Convert "success" and "failure" to binary and "nonexistent" to null values
campaign['previous_outcome'] = campaign['previous_outcome'].map({
    'success': 1,
    'failure': 0,
    'nonexistent': np.nan
})
campaign['campaign_outcome'] = campaign['campaign_outcome'].map({'yes': 1, 'no': 0})

# Add a column called campaign_id in campaign
campaign['campaign_id'] = 1

C:\Users\rubos\AppData\Local\Temp\ipykernel_18044\1906349684.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client['education'] = client['education'].str.replace('.', '_')
C:\Users\rubos\AppData\Local\Temp\ipykernel_18044\1906349684.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client['job'] = client['job'].str.replace('.', '')


In [3]:
# Convert month abbreviations to integers
month_dict = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
campaign['month'] = campaign['month'].map(month_dict)

# Map day_of_week abbreviations to integers
day_dict = {
    'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5, 'sat': 6, 'sun': 7
}
campaign['day_of_week'] = campaign['day_of_week'].map(day_dict)

# Create a datetime column called last_contact_date
campaign['last_contact_date'] = pd.to_datetime(2022 * 10000 + campaign['month'] * 100 + campaign['day_of_week'], format='%Y%m%d')

# Remove redundant data
campaign = campaign.drop(columns=['contact', 'month', 'day_of_week'])

In [4]:
# Save the three DataFrames to csv files without an index
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

In [5]:
# SQL code to create tables
client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    contact_duration INTEGER,
    number_contacts INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """
CREATE TABLE economics (
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    cons_conf_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""